In [44]:
from keys import google_maps
import requests 
import json
import pandas as pd
from pandas.io.json import json_normalize

In [76]:
search_params = {'input':'Bellevue%20University','type': 'textquery', 'fields':'place_id', 'api_key': secret['api_key']}

In [99]:
def find_place(params):
    '''
    Searches for a place and returns the place ID. 
    Params:
        a dict of the following: 
        input: term which will be searched
        type: type of search to be performed
        fields: data to return back from search (always place_id)
        api_key: secret api key to make the request
    '''
    url = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=_IN_&inputtype=_TP_&fields=_F_&key=_K_'
    url = url.replace('_IN_', params['input']).replace('_TP_',params['type']).replace('_F_',params['fields']).replace('_K_',secret['api_key'])
    r = requests.get(url)
    result = r.json()
    candidates = result['candidates'][0]
    return candidates['place_id']    

In [102]:
place_id = find_place(search_params)

In [113]:
find_place_by_id_params = {'fields':'rating,formatted_address,geometry,name,permanently_closed,place_id,plus_code,scope,type,url,vicinity,website,formatted_phone_number,international_phone_number',
                           'placeid':place_id,
                           'api_key': secret['api_key']
                          }

In [114]:
def get_place_by_id(params_dict):
    '''
    Searches for place once the ID is known. The function also returns a dataframe.
    The dataframe will serve as the first data cleanup step by transforming raw JSON
    into a dataframe
    Params
    '''
    url = 'https://maps.googleapis.com/maps/api/place/details/json?placeid=_P_&fields=_F_&key=_K_'
    url = url.replace('_P_', params_dict['placeid']).replace('_F_',params_dict['fields']).replace('_K_',params_dict['api_key'])
    r = requests.get(url)
    r_text = r.text
    r_text= json.loads(r_text)
    df = json_normalize(r_text)
    return df
    

In [115]:
df = get_place_by_id(find_place_by_id_params)

In [116]:
df

,html_attributions,result.formatted_address,result.formatted_phone_number,result.geometry.location.lat,result.geometry.location.lng,result.geometry.viewport.northeast.lat,result.geometry.viewport.northeast.lng,result.geometry.viewport.southwest.lat,result.geometry.viewport.southwest.lng,result.international_phone_number,...,result.place_id,result.plus_code.compound_code,result.plus_code.global_code,result.rating,result.scope,result.types,result.url,result.vicinity,result.website,status
0,[],"1000 Galvin Rd S, Bellevue, NE 68005, USA",(402) 293-2000,41.150977,-95.919596,41.155714,-95.915233,41.146619,-95.92562,+1 402-293-2000,...,ChIJf4DKhmqIk4cR63XRQHKFjoM,"532J+95 Bellevue, NE, United States",86H6532J+95,3.8,GOOGLE,"[university, point_of_interest, establishment]",https://maps.google.com/?cid=9479660991421707755,"1000 Galvin Road South, Bellevue",http://www.bellevue.edu/,OK


## Data Cleanup #2
Drop bad columns and rename

In [124]:
df = df.drop(columns=['html_attributions','status'])

In [125]:
df.columns

Index(['result.formatted_address', 'result.formatted_phone_number',
       'result.geometry.location.lat', 'result.geometry.location.lng',
       'result.geometry.viewport.northeast.lat',
       'result.geometry.viewport.northeast.lng',
       'result.geometry.viewport.southwest.lat',
       'result.geometry.viewport.southwest.lng',
       'result.international_phone_number', 'result.name', 'result.place_id',
       'result.plus_code.compound_code', 'result.plus_code.global_code',
       'result.rating', 'result.scope', 'result.types', 'result.url',
       'result.vicinity', 'result.website'],
      dtype='object')

In [129]:
columns_dict = {}
for column in df.columns:
    columns_dict[column] = column.replace('result.','')
df = df.rename(columns=columns_dict)